In [1]:
stock = 'NVDA'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    249.273651
2018-07-24    248.574036
2018-07-25    251.732300
2018-07-26    254.700684
2018-07-27    251.882233
2018-07-30    243.996536
2018-07-31    244.726135
2018-08-01    246.335251
2018-08-02    250.482986
2018-08-03    251.962189
2018-08-06    253.891129
2018-08-07    256.809540
2018-08-08    258.278748
2018-08-09    256.319794
2018-08-10    254.650696
2018-08-13    255.979980
2018-08-14    261.287079
2018-08-15    258.938354
2018-08-16    257.299255
2018-08-17    244.686172
2018-08-20    247.704498
2018-08-21    253.181519
2018-08-22    262.676331
2018-08-23    266.694122
2018-08-24    272.071167
2018-08-27    275.749176
2018-08-28    274.230011
2018-08-29    278.489990
2018-08-30    277.809998
2018-08-31    280.679993
                 ...    
2018-09-18    271.019989
2018-09-19    271.980011
2018-09-20    266.279999
2018-09-21    263.450012
2018-09-24    265.700012
2018-09-25    268.410004
2018-09-26    266.920013
2018-09-27    267.399994
2018-09-28    281.01

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


227.2591496345526
[232.93337517 225.68650018 218.45980105 219.88273461 239.33333717]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")